In [1]:
import json
from opencc import OpenCC
import requests

# 读取并返回 JSON 数据
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)
    
# 格式化 JSON 数据为字符串
def format_json(data):
    return json.dumps(data, ensure_ascii=False, indent=4)

# 从指定 URL 获取 JSON 并返回解析后的 Python 对象
def fetch_json(url):
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.json()

# 将 JSON 数据写入文件
def write_json(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [2]:
tw_url = 'https://sg-tools-cdn.blablalink.com/vm-36/bj-70/6223a9fbfd3be53b48587c934a91f686.json'
en_url = 'https://sg-tools-cdn.blablalink.com/yl-57/hd-03/1bf030193826e243c2e195f951a4be00.json'


tw_data = fetch_json(tw_url)
en_data = fetch_json(en_url)

# 初始化繁简转换器
cc = OpenCC('t2s')  # 将繁体转换为简体

# 将tw_data中的繁体文本转换为简体
def convert_to_simplified(data):
    if isinstance(data, dict):
        return {key: convert_to_simplified(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_simplified(item) for item in data]
    elif isinstance(data, str):
        return cc.convert(data)
    else:
        return data

tw_data_simplified = convert_to_simplified(tw_data)

# 输出原始数据到json文件 (可选，如果不再需要可以注释掉)
# write_json('tw_data.json', tw_data_simplified)
# write_json('en_data.json', en_data)

en_dict = { entry['id']: entry for entry in en_data }

nikkes_list = []
for tw in tw_data_simplified:
    uid = tw['id']
    if uid not in en_dict:
        # 如果英文表里没有对应的 id，可以选择跳过或保留部分字段；这里跳过
        continue

    en = en_dict[uid]

    # 提取各字段
    item = {
        'id': uid,
        'name_code': tw.get('name_code'),
        'class': tw.get('class'),
        # 现在已经是简体中文了
        'name_cn': tw['name_localkey']['name'],
        'name_en': en['name_localkey']['name'],
        'element': tw['element_id']['element']['element'],
        'use_burst_skill': tw.get('use_burst_skill'),
        'corporation': tw.get('corporation'),
        'weapon_type': tw['shot_id']['element']['weapon_type'],
        'original_rare': tw.get('original_rare'),
    }
    nikkes_list.append(item)

# 自定义的 cubes 数据
cubes_data = [
    {
        "cube_id": 1000301, 
        "name_cn": "遗迹突击魔方",
        "name_en": "Assault Cube"
    },
    {
        "cube_id": 1000302, 
        "name_cn": "战术突击魔方",
        "name_en": "Onslaught Cube"
    },
    {
        "cube_id": 1000303,
        "name_cn": "遗迹巨熊魔方",
        "name_en": "Resilience Cube"
    },
    {
        "cube_id": 1000304,
        "name_cn": "战术巨熊魔方",
        "name_en": "Bastion Cube"
    },
    {
        "cube_id": 1000305,
        "name_cn": "遗迹促进魔方",
        "name_en": "Adjutant Cube"
    },
    {
        "cube_id": 1000306,
        "name_cn": "战术促进魔方",
        "name_en": "Wingman Cube"
    },
    {
        "cube_id": 1000307,
        "name_cn": "遗迹量子魔方",
        "name_en": "Quantum Cube"
    },
    {
        "cube_id": 1000308,
        "name_cn": "体力神器魔方",
        "name_en": "Vigor Cube"
    },
    {
        "cube_id": 1000309,
        "name_cn": "遗迹强韧魔方",
        "name_en": "Endurance Cube"
    },
    {
        "cube_id": 1000310,
        "name_cn": "遗迹治疗魔方",
        "name_en": "Healing Cube"
    },
    {
        "cube_id": 1000311,
        "name_cn": "遗迹回火魔方",
        "name_en": "Tempering Cube"
    },
    {
        "cube_id": 1000312,
        "name_cn": "遗迹辅助魔方",
        "name_en": "Relic Assist Cube"
    },
    {
        "cube_id": 1000313,
        "name_cn": "遗迹毁灭魔方",
        "name_en": "Destruction Cube"
    },
    {
        "cube_id": 1000314,
        "name_cn": "遗迹穿透魔方",
        "name_en": "Piercing Cube"
    }
]

# 组合最终的 JSON 结构
final_data = {
    "cubes": cubes_data,
    "nikkes": nikkes_list
}

output_path = 'list.json' # 输出到根目录的 list.json
# 如果需要输出到 public 目录，路径应为 'Extension/exia-invasion/public/list.json'
# output_path = 'Extension/exia-invasion/public/list.json' 
write_json(output_path, final_data)

print(f"数据已成功写入到 {output_path}")

数据已成功写入到 list.json
